# Hw9

We have to use Deep RL in this HW because if we use normal RL it will not converge to global minima. <br>
Because of two major reasons: <br>
1. correlation between samples
2. Non-stationary targets


##### Because of the non-stationary targets problem, I didn't use the code given in demo. In our Neural Network problem, we are trying to predict the current state using future state.

In [1]:
import gym
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

The maximum goal_steps that we can keep is 200 because that is the contraint of the game. We need to reach the goal by those many epoches. we text this 10000 times that why inital_games is 100000. score_requirement is -198 because we want to keep score towards the right and not the left. as right side has rewards

In [2]:
env = gym.make('MountainCar-v0')
env.reset()
goal_steps = 200
score_requirement = -198
intial_games = 10000

A particular epoch finishes when top(0.5) position is reached or 200 iterations are finished.  The code was not able to converge to optimal points and the success rate was very poor every time. In order to avoid this, few changes were made to the main program data population. 

In [15]:
def pre_process_data():
    acc_scores = []
    train_data = []
    for game_index in range(intial_games):
        score_game = 0
        game_memory = []
        prev_obs = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 3)
            observation, reward, done, info = env.step(action)
            
            if len(prev_obs) > 0:
                game_memory.append([prev_obs, action])
                
            prev_obs = observation
            if observation[0] > -0.2:
                reward = 1
            
            score_game += reward
            if done:
                break
            
        if score_game >= score_requirement:
            acc_scores.append(score_game)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1, 0]
                elif data[1] == 0:
                    output = [1, 0, 0]
                elif data[1] == 2:
                    output = [0, 0, 1]
                train_data.append([data[0], output])
        
        env.reset()
    
    print(acc_scores)
    
    return train_data

The rewards given by the environment were taken as 1 when the oberavation was above -0.2. This was done as it could've been the top of the hill which means reward should be given. Now coming to the problem of converging to local minima, after every action, the agent will take negative rewards so that we don't reach the minima. 

In [16]:
train_data = pre_process_data()

[-176.0, -188.0, -192.0, -186.0, -188.0, -178.0, -192.0, -176.0, -182.0, -190.0, -166.0, -192.0, -174.0, -176.0, -190.0, -192.0, -184.0, -166.0, -190.0, -182.0, -172.0, -190.0, -188.0, -194.0, -170.0, -166.0, -164.0, -168.0, -164.0, -180.0, -164.0, -178.0, -164.0, -192.0, -184.0, -182.0, -194.0, -194.0, -198.0, -186.0, -172.0, -180.0, -178.0, -182.0, -170.0, -182.0, -194.0, -194.0, -198.0, -192.0, -182.0, -176.0, -192.0, -192.0]


In [17]:
def build_model(input_size, output_size):
        model = Sequential()
        model.add(Dense(256, input_dim=input_size, activation='relu'))
        model.add(Dense(128, input_dim=input_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(output_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam())

        return model

In [18]:
def train_model(train_data):
    X = np.array([i[0] for i in train_data]).reshape(-1, len(train_data[0][0]))
    y = np.array([i[1] for i in train_data]).reshape(-1, len(train_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model

In [19]:
trained_model = train_model(train_data)

Epoch 1/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2247
Epoch 2/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2217
Epoch 3/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2203
Epoch 4/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2203
Epoch 5/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2196
Epoch 6/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2193
Epoch 7/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2193
Epoch 8/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2192
Epoch 9/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2193
Epoch 10/10
336/336 [==============================] - 1s 2ms/step - loss: 0.2192


In [20]:
choices = []
scores = []

for each_game in range(100):
    score = 0
    prev_obs = []
    for step in range(goal_steps):
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{} choice 2:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices),choices.count(2)/len(choices)))

[-101.0, -123.0, -151.0, -122.0, -129.0, -119.0, -119.0, -97.0, -120.0, -119.0, -129.0, -131.0, -91.0, -129.0, -175.0, -129.0, -174.0, -126.0, -127.0, -130.0, -111.0, -129.0, -120.0, -176.0, -147.0, -147.0, -137.0, -124.0, -119.0, -124.0, -97.0, -125.0, -93.0, -121.0, -143.0, -124.0, -122.0, -181.0, -125.0, -130.0, -131.0, -130.0, -118.0, -91.0, -123.0, -149.0, -183.0, -107.0, -131.0, -150.0, -121.0, -131.0, -121.0, -120.0, -119.0, -94.0, -175.0, -120.0, -106.0, -130.0, -131.0, -119.0, -150.0, -135.0, -119.0, -91.0, -129.0, -132.0, -134.0, -119.0, -151.0, -120.0, -130.0, -129.0, -129.0, -101.0, -129.0, -120.0, -95.0, -129.0, -130.0, -129.0, -130.0, -137.0, -174.0, -150.0, -120.0, -137.0, -130.0, -137.0, -183.0, -91.0, -118.0, -131.0, -111.0, -124.0, -128.0, -174.0, -119.0, -130.0]
Average Score: -128.61
choice 1:0.12028613638130783  choice 0:0.3150610372443822 choice 2:0.5646528263743099


In [21]:
print ("Success rate: " +  str(abs(sum(scores)/goal_steps)))

Success rate: 64.305
